# Drum Transcriber Colab

Run this notebook to launch the Drum Transcriber Web UI.

In [ ]:
# @title 1. Setup Environment
# @markdown This cell installs necessary libraries and clones the repository (if not already present).

import os

repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber"

if os.path.exists(repo_name):
    %cd {repo_name}
    !git pull
else:
    !git clone {repo_url}
    %cd {repo_name}

!pip install gradio yt-dlp librosa tensorflow pandas numpy plotly scikit-learn soundfile


In [ ]:
# @title 2. Download Model
# @markdown The model file in the repo might be a git-lfs pointer or corrupted. We download it directly from the source provided in the README.

import gdown
import os

model_id = '1w2fIHeyr-st3sbk1PYrtGOYW6YAD1fsi'
output_path = 'model/drum_transcriber.h5'
url = f'https://drive.google.com/uc?id={model_id}'

if os.path.exists(output_path):
    print("Removing existing model file to ensure fresh download...")
    os.remove(output_path)

# Ensure the model directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

print("Downloading model...")
gdown.download(url, output_path, quiet=False)
print("Model downloaded.")


In [ ]:
# @title 3. Run App
# @markdown This will launch the Gradio interface. Click the public URL (e.g., `https://xxxx.gradio.live`) to use it.

!python gradio_app.py
